# Kaggle Caption Notebook

Script dành riêng cho Kaggle để tối ưu thời gian xử lý hình ảnh. 
Chỉ xử lý 2 video mỗi lần để tận dụng hiệu quả thời gian chạy 12 giờ của Kaggle.

## 1. Cài đặt thư viện

In [ ]:
# Cài đặt transformers từ source để đảm bảo phiên bản mới nhất
!pip install git+https://github.com/huggingface/transformers

In [ ]:
# Cài đặt bitsandbytes để hỗ trợ quantization
!pip install -U bitsandbytes -q

## 2. Tải dữ liệu keyframes

In [ ]:
# THAY ĐỔI BATCH Ở ĐÂY
!gdown 14MeYV2WBWwldMDGRrpG9s7vz8triwbWr
!unzip -qq L01.zip
!rm -rf L01.zip

## 3. Cấu hình video để xử lý

In [ ]:
# Chọn video để xử lý
# START_VIDEO_INDEX: chỉ số bắt đầu (0 cho V001/V002, 2 cho V003/V004, ...)
# VIDEO_BATCH_SIZE: số video xử lý mỗi lần (mặc định là 2)

import sys, os

# Thiết lập cấu hình xử lý
BATCH_L = "L01"  # Đường dẫn đến thư mục batch
START_VIDEO_INDEX = 2  # Đổi thành 0, 2, 4, ... tùy theo video cần xử lý
VIDEO_BATCH_SIZE = 2   # Xử lý 2 video mỗi lần

print(f"Xử lý batch: {BATCH_L}")
print(f"Bắt đầu từ video index: {START_VIDEO_INDEX}")
print(f"Số lượng video xử lý: {VIDEO_BATCH_SIZE}")

## 4. Import thư viện

In [ ]:
import torch
import os
import json
import glob
from tqdm import tqdm
from transformers import AutoProcessor, AutoModelForImageTextToText, BitsAndBytesConfig

## 5. Cấu hình model

In [ ]:
MODEL_CHECKPOINT = "OpenGVLab/InternVL3-8B-hf"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu" 
print(f"\nUsing device: {DEVICE}")
DATA_TYPE = torch.bfloat16
MAX_NEW_TOKENS = 100

## 6. Tải model và processor

In [ ]:
# Attempt to use 4-bit quantization on GPU if available
quantization_config = None
if DEVICE == "cuda":
    try:
        quantization_config = BitsAndBytesConfig(load_in_4bit=True)
        print("Using 4-bit quantization")
    except Exception as e:
        print(f"bitsandbytes not available or incompatible → loading full-precision model. Details: {e}")

processor = AutoProcessor.from_pretrained(MODEL_CHECKPOINT)

model_kwargs = dict(torch_dtype=DATA_TYPE)
if quantization_config is not None:
    model_kwargs["quantization_config"] = quantization_config

model = AutoModelForImageTextToText.from_pretrained(MODEL_CHECKPOINT, **model_kwargs).eval().to(DEVICE)

## 7. Định nghĩa prompt và hàm caption

In [ ]:
PROMPT = (
    "You are an image captioning assistant processing still frames from live broadcast news videos. "
    "Provide a concise but informative description that mentions the main subjects (people or objects), "
    "their actions, and the scene context. "
    "Ignore any on-screen graphics such as ticker text, news banners, program logos, watermarks, "
    "or other unrelated overlays."
)

def describe_keyframe(image_path):
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image_path},
                {"type": "text", "text": PROMPT}
            ]
        }
    ]
    
    inputs = processor.apply_chat_template(messages, add_generation_prompt=True, tokenize=True, return_dict=True, return_tensors="pt").to(model.device, dtype=DATA_TYPE)
    output = model.generate(**inputs, max_new_tokens=MAX_NEW_TOKENS, do_sample=False)
    answer = processor.decode(output[0, inputs["input_ids"].shape[1]:], skip_special_tokens=True).strip()
    return answer

## 8. Xử lý 2 video được chỉ định

In [ ]:
# Lấy danh sách tất cả các video
all_videos = sorted(glob.glob(os.path.join(BATCH_L, "V*")))

# Chọn video cần xử lý dựa trên chỉ số bắt đầu và số lượng
videos_to_process = all_videos[START_VIDEO_INDEX:START_VIDEO_INDEX + VIDEO_BATCH_SIZE]

if not videos_to_process:
    print(f"Không tìm thấy video nào bắt đầu từ index {START_VIDEO_INDEX}!")
else:
    print(f"Xử lý {len(videos_to_process)} video:")
    for v in videos_to_process:
        print(f" - {os.path.basename(v)}")
    
    # Tạo thư mục results nếu chưa tồn tại
    os.makedirs("results", exist_ok=True)
    
    # Xử lý từng video được chọn
    for video_dir in videos_to_process:
        video_name = os.path.basename(video_dir)
        keyframes = sorted(glob.glob(os.path.join(video_dir, "*.jpg")))
        
        if not keyframes:
            print(f"CẢNH BÁO: Không tìm thấy keyframe nào trong {video_dir}")
            continue
            
        print(f"\nĐang xử lý {video_name} với {len(keyframes)} keyframes...")
        
        video_results = []
        for keyframe_path in tqdm(keyframes, desc=video_name):
            keyframe_name = os.path.basename(keyframe_path)
            caption = describe_keyframe(keyframe_path)
            video_results.append({"keyframe": keyframe_name, "caption": caption})
        
        # Lưu kết quả tùng video thành file JSON riêng
        output_file = os.path.join("results", f"{BATCH_L}_{video_name}_caption.json")
        try:
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(video_results, f, indent=2, ensure_ascii=False)
            print(f"✓ Đã lưu kết quả vào: {output_file}")
        except Exception as e:
            print(f"✗ Lỗi khi lưu kết quả vào {output_file}: {e}")

## 9. Làm sạch thư mục tạm sau khi xử lý

In [ ]:
!rm -rf {BATCH_L}